In [215]:
# Установка библиотек

# pip install lightfm

In [216]:
# Импорт библиотек

import os # Для взаимодействия с ОС
import re # Для работы с регулярными выражениями
import shutil # Для управления файлами и папками
import numpy as np # Для работы с массивами и математическими функциями
import pandas as pd # Для работы с табличными данными
from itertools import product # Для создания декартова произведения итераторов
from lightfm import LightFM # Для реализации алгоритмов рекомендательных систем
from lightfm.data import Dataset # Для рабоы с табличными данными
from scipy.sparse import coo_matrix # Для работы с разряжёнными матрицами
from sklearn.metrics import precision_score # Для вычисоления метрики точнсти модели машинного обучения
from lightfm.evaluation import precision_at_k # Для вычисоления метрики точнсти модели машинного обучения
from lightfm.cross_validation import random_train_test_split # Для разделения на обучающую и тестовую выборки

## Copying to the Repository

In [217]:
# Функция копирования файлов из коренной директории в директорию с репозиторием

def copying_to_repository():
    # Путь к коренной директории и директории с репозиторием
    source_folder = input('Введите коренную директорию - ')
    git_folder = input('Введите директорию репозитория - ')

    # Создание целевой директории
    os.makedirs(git_folder, exist_ok = True)

    # Копирование файлов из исходной директории в репозиторий
    for filename in os.listdir(source_folder):
        # Полный путь к файлу в корневой директории
        source_path = os.path.join(source_folder, filename)
        # Полный путь к директории с репозиторием
        git_path = os.path.join(git_folder, filename)

        # Исключение папки при копировании
        if filename != '.ipynb_checkpoints':
            # Если это директория, то копируем её с содержимым
            if os.path.isdir(source_path):
                # Удаление старой директории
                shutil.rmtree(git_path, ignore_errors = True)
                # Копирование директории
                shutil.copytree(source_path, git_path)
            else:
                # Копирование файла
                shutil.copy2(source_path, git_path)

    print('Файлы успешно скопированы в репозиторий')

In [218]:
# Приминение функции копирования файлов из коренной директории в директорию с репозиторием

# copying_to_repository()

## Data Preparation

### Data preparation - Characteristic and sales of goods

In [219]:
# Загрузка датасета
# df_chatacteristic = pd.read_excel('data/characteristics_and_sales_of_goods.xlsx')

# Вывод размера датасета
# print(f'Размер датасета: {df_chatacteristic.shape}')

# Вывод первых пять записей датасета
# print('Первые пять записей датасета')
# df_chatacteristic.head()

In [220]:
# Функция обработки датасета с характеристиками товаров

def preparation_goods_characteristics_data(df):
    # Привидение текстовых переменных к нижнему регистру
    df[['product_name', 'product_category']] = df[['product_name', 'product_category']].apply(lambda x: x.str.lower())

    # Словарь для объединения категорий
    category_mapping = {
        'конфеты': ['конфеты', 'конфеты фасованные', 'конфеты в коробках'],
        'шоколад': ['шоколад', 'шоколад весовой', 'шоколодные и ореховые пасты'],
        'печенье': ['печенье', 'печенье фасованное', 'крекер'],
        'вафли': ['вафли', 'вафли фасованные'],
        'карамель': ['карамель', 'карамель фасованная'],
        'чай': ['чай', 'хорека-чай'],
        'кофе': ['кофе', 'хорека-кофе'],
        'консервы': ['консервы мясные', 'консервы овощные', 'консервы рыбные и морепродукты', 'консервы фруктово-ягодные'],
        'молоко сгущенное': ['молоко сгущенное и концентрированное'],
        'мясные продукты': ['деликатесы мясные', 'мясо', 'мясо охлажденное', 'холодец'],
        'полуфабрикаты': ['полуфабрикаты', 'готовые блюда', 'паназиатская кухня'],
        'хлебобулочные изделия': ['хлеб, хлебобулочные изделия'],
        'к чаю': ['баранки, соломка, сушка, сухари', 'кексы', 'рулеты', 'зефир', 'мармелад', 'пастила', 'восточные сладости', 'козинаки', 'халва'],
        'десерты': ['десерты', 'торты ,бисквиты,пирожные', 'хорека-коржи, торты'],
        'напитки': ['соки, напитки'],
        'крупы и зерновые': ['крупы', 'каши'],
        'растительные масла': ['масла растительные'],
        'соусы и приправы': ['кетчуп и соусы', 'томатная паста'],
        'приправы': ['специи, приправы', 'смеси для гарнира'],
        'продукты для выпечки': ['сода, уксус, дрожжи'],
        'сухие завтраки': ['сухие завтраки', 'хлопья'],
        'замороженные продукты': ['замороженные овощи, ягоды, грибы'],
        'овощи/фрукты': ['овощи,фрукты грибы,ягоды'],
        'детские товары': ['детское питание', 'детские товары'],
        'полезное питание': ['полезное питание', 'полезное питание , продукция без сахара'],
        'снэки': ['снэки', 'жевательная резинка'],
        'новогодние товары': ['новогодние подарки нк', 'новогодние подарки', 'новогодняя упаковка/сайт', 'новогодняя упаковка'],
        'прочее': ['прочие', 'прочие нк(швейный цех)', 'нпу 2024', 'янеиспользуемый', 'остатки сладки', 'элитная продукция', 'подарки'],
        'товары для дома': ['хозяйственные товары', 'бытовая химия'],
        'личная гигиена': ['личная гигиена', 'косметика, гигиена'],
        'табак': ['табак, спички', 'табак собственные магазины'],
        'рыба': ['рыба', 'рыбные пресервы'],
        'яйца': ['яйцо'],
        'орехи/сухофрукты': ['орехи, сухофрукты'],
        'животные': ['товары для животных'],
        'собственные магазины': ['собственные магазины']
    }

    # Функция для объединения категорий товаров
    def replace_categories(df, column_name):
      # Обратный словарь для замены, где ключ - подкатегория, значение - основная категория
      reverse_mapping = {}
      # Заполнение обратного словаря
      # Проход по категориям товаров
      for categories, subcategories in category_mapping.items():
          # Проход по подкатегориям категории товара
          for subcategory in subcategories:
              # Заполнения обратного словаря категорями товаров на основе подкатегории товаров
              reverse_mapping[subcategory] = categories
      # Заполнение стобца датасета категориями из обратного словаря
      df.loc[:, column_name] = df[column_name].map(reverse_mapping).fillna(df[column_name])
      return df

    # Приминение функции замены категорий товаров
    df = replace_categories(df, 'product_category')

    # Удаление товаров с отрицательной суммой реализации
    df = df.loc[df['total_realization'] > 0].copy()

    # Удаление категорий товаров у которых количество товаров меньше 10
    # Подсчёт количества товаров по категориям
    category_counts = df['product_category'].value_counts()
    # Выбор категорий с менее чем 10 товарами
    rare_categories = category_counts[category_counts < 10].index
    # Удаление строк с редкими категориями из датасета
    df = df.loc[~df['product_category'].isin(rare_categories)].copy()

    # Преобразования накопительной суммы реализации в проценты
    df['accumulation_realization'] = df['accumulation_realization'].apply(lambda x: round((x * 100), 2))

    # Функция ABC-классификации товаров
    def abc_classification(accumulation_realization):
        if accumulation_realization < 80:
            return 'A'
        elif 80 <= accumulation_realization <= 95:
            return 'B'
        else:
            return 'C'

    # Создание нового столбца с ABC-классификацией товаров
    df['abc_classification'] = df['accumulation_realization'].apply(abc_classification)

    # Удаление столбцов датасета
    df = df.drop(['share_realization', 'accumulation_realization'], axis = 1)

    # Возврат обработанного датасета с характеристиками товаров
    return df

In [221]:
# Применение функция обработки датасета с характеристиками товаров
# df_chatacteristic = preparation_goods_characteristics_data(df_chatacteristic)

# Сохранение датасета
# df_chatacteristic.to_excel('data/characteristics_and_sales_of_goods_prepared.xlsx')

### Data preparation - Sales of goods by store

In [222]:
# Загрузка датасета
# df_sales = pd.read_excel('data/sales_of_goods_by_store.xlsx')

# Вывод размера датасета
# print(f'Размер датасета: {df_sales.shape}')

# Вывод первых пять записей датасета
# print('Первые пять записей датасета')
# df_sales.head()

In [223]:
# Функция создания матрицы количества проданных товаров по торговым точкам

def creating_googs_sales_matrix(df):
    # Переименовывание колонок
    df.columns = ['raw_text', 'quantity']

    # Список для хранения названий торговых точек
    store_names = []
    # Переменная для названия торговой точки
    current_store = None

    # Перебор всех строк в датасете
    for i, row in df.iterrows():
        # Если в строке 'quantity' — NaN, значит это строка с названием торговой точки
        if pd.isna(row['quantity']):
            # Записывание в переменную название торговой точки, убираем лишние символы
            current_store = str(row['raw_text'])
        # Добавление переменной с торговой точкой в список
        store_names.append(current_store)

    # Добавление названий торговых точек в столбец датасета
    df['store_name'] = store_names
    # Фильтрация только строк с товарами
    df = df[pd.notna(df['quantity'])].copy()
    # Преобразование 'quantity' в тип данных float
    df['quantity'] = df['quantity'].apply(lambda x: float(str(x).replace(",", ".")))
    # Переименовывание колонки с названием товара
    df = df.rename(columns = {'raw_text': 'product_name'})
    # Сброс индекса после фильтрации
    df = df.reset_index(drop = True)
    # Привидение текстовых переменных к нижнему регистру
    df[['product_name', 'store_name']] = df[['product_name', 'store_name']].apply(lambda x: x.str.lower())

    # Функция для извлечения названия торговой точки из столбца 'store_name'
    def extract_store_name(store_name):
        # Поиск любого содержания в кавычках
        match = re.search(r'"([^"]*)"', store_name)
        # Возвращаем найденный текст
        return match.group(1) if match else None

    # Применение изменений в столбце 'store_name'
    df['store_name'] = df['store_name'].apply(extract_store_name)

    # Функция для фильтрации по длине слов в названии торговой точки из столбца 'store_name'
    def deleting_rows_by_store(store_name):
        # Проверка на пропущенные значения
        if pd.isna(store_name):
            return False
        # Разбиение слов
        words = str(store_name).split()
        # Условие удаления строк
        return all(5 <= len(word) <= 15 for word in words)

    # Применение изменений в столбце 'store_name'
    df = df[df['store_name'].apply(deleting_rows_by_store)]

    # Группировка по торговым точкам и товарам, суммируя 'quantity'
    df = df.groupby(['store_name', 'product_name'])['quantity'].sum().reset_index()
    # Подсчёт количества уникальных товаров для каждой торговой точки
    number_unique_products = df.groupby('store_name')['product_name'].nunique().reset_index()
    number_unique_products.columns = ['store_name', 'unique_products']

    # Фильтрация торговых точек по количеству уникальных товаров
    filtering_stores_by_number_unique_products = number_unique_products[(number_unique_products['unique_products'] >= 150) & (number_unique_products['unique_products'] <= 1000)]
    # Выбор 100 случайных торговых точек
    filtere_stores = filtering_stores_by_number_unique_products.sample(n = min(150, len(filtering_stores_by_number_unique_products)), random_state = 42) if not filtering_stores_by_number_unique_products.empty else filtering_stores_by_number_unique_products
    # Фильтрация датасета по выбранным торговым точкам
    df = df[df['store_name'].isin(filtere_stores['store_name'])]

    # Создание матрицы количества проданных товаров по торговым точкам
    df = df.pivot_table(index = 'product_name', columns = 'store_name', values = 'quantity', fill_value = 0)

    # Возврат матрицы количества проданных товаров по торговым точкам
    return df

In [224]:
# Применение функции создания матрицы количетсва проданных товаров по торговым точкам
# df_sales = creating_googs_sales_matrix(df_sales)

# Сохранение датасета
# df_sales.to_excel('data/matrix_number_goods_sold_in_stores.xlsx')

### Data preparation - Merge datasets

In [225]:
# Загрузка датасета с матрицей количетсва проданных товаров по торговым точкам
df_matrix = pd.read_excel('data/matrix_number_goods_sold_in_stores.xlsx', index_col = 0)

# Вывод размера датасета
print(f'Размер датасета: {df_matrix.shape}')

# Вывод первых пять записей датасета
print('Первые пять записей датасета')
df_matrix.head()

Размер датасета: (6679, 150)
Первые пять записей датасета


,24/25,аванд,адонис-библ,александра,аленка,алисон,аллея,алмаз,алмат,алтон,...,шодруз,шумковский,экспресс,элита,эскадра,юбилейный,южный,яблоко,яблоня,яранга
product_name,,,,,,,,,,,,,,,,,,,,,
fantola молочный с начинкой вкус голубая малина и печеньем 66г,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fantola молочный с начинкой со вкусом avocado-fest и печеньем 66г,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fantola молочный с начинкой со вкусом bubble gum и печеньем 66г,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fantola молочный с начинкой со вкусом choco vibe c начинкой и печеньем 66г,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"шедевр хрустящие со слив.начинкой, целым фундуком темный шок. 145г",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [226]:
# Загрузка датасета с информацией о товарах
df_info = pd.read_excel('data/characteristics_and_sales_of_goods_prepared.xlsx', index_col=0)

# Вывод размера датасета
print(f'Размер датасета: {df_info.shape}')

# Вывод первых пять записей датасета
print('Первые пять записей датасета')
df_info.head()

Размер датасета: (20064, 4)
Первые пять записей датасета


,product_name,product_category,total_realization,abc_classification
0,аленка (ок) 90г,шоколад,71201224.45,A
1,роллтон вермишель б/п витамин. куриная на дома...,продукты быстрого приготовления,64360762.22,A
2,"макфа мука пшеничная в/с, 2кг.челябинск",мука,49262315.82,A
3,"макфа спираль макар.изд., в/с, 223-3а 400гр.",макаронные изделия,47573899.91,A
4,в шок. москвичка бк,карамель,32893871.30,A


In [227]:
# Объединение датасетов по столбцу 'product_name'
df_merged = df_matrix.merge(df_info[['product_name', 'product_category', 'abc_classification', 'total_realization']], left_index = True, right_on = 'product_name', how = 'right')

# Вывод размера датасета
print(f'Размер датасета: {df_merged.shape}')

# Вывод первых пять записей датасета
print('Первые пять записей датасета')
df_merged.head()

Размер датасета: (20064, 154)
Первые пять записей датасета


,24/25,аванд,адонис-библ,александра,аленка,алисон,аллея,алмаз,алмат,алтон,...,эскадра,юбилейный,южный,яблоко,яблоня,яранга,product_name,product_category,abc_classification,total_realization
0,0.0,0.0,5.0,10.0,75.0,30.0,6390.0,15.0,0.0,15.0,...,0.0,20.0,70.0,45.0,15.0,60.0,аленка (ок) 90г,шоколад,A,71201224.45
1,0.0,0.0,0.0,84.0,378.0,210.0,0.0,336.0,168.0,0.0,...,0.0,0.0,210.0,84.0,84.0,42.0,роллтон вермишель б/п витамин. куриная на дома...,продукты быстрого приготовления,A,64360762.22
2,12.0,0.0,0.0,0.0,24.0,0.0,0.0,24.0,12.0,0.0,...,0.0,0.0,12.0,0.0,6.0,0.0,"макфа мука пшеничная в/с, 2кг.челябинск",мука,A,49262315.82
3,4.0,5.0,0.0,0.0,3.0,2.0,3040.0,7.0,4.0,0.0,...,0.0,24.0,5.0,3.0,0.0,5.0,"макфа спираль макар.изд., в/с, 223-3а 400гр.",макаронные изделия,A,47573899.91
4,0.0,0.0,0.0,2.0,9.0,2.0,0.0,13.0,1.0,0.0,...,0.0,5.0,7.0,2.0,0.0,3.0,в шок. москвичка бк,карамель,A,32893871.30


In [228]:
# Заполнение пропуенных значений в матрице нулями
df_merged = df_merged.fillna(0)

# Вывод пропущенных значений в датасете
print('Вывод пропущенных значений в датасете:')
df_merged.isnull().sum()

Вывод пропущенных значений в датасете:


,0
24/25,0
аванд,0
адонис-библ,0
александра,0
аленка,0
...,...
яранга,0
product_name,0
product_category,0
abc_classification,0


## Modeling

In [229]:
# Создание объекта Dataset для хранения данных
df_lightfm = Dataset()

In [230]:
# Регистрация торговых точек и товаров
df_lightfm.fit(users = df_matrix.columns, items = df_merged['product_name'])

In [231]:
# Формирования списка уникальных признаков товара (категория + ABC-классификация)
goods_feature_list = (df_merged['product_category'] + ' ' + df_merged['abc_classification']).unique().tolist()
# Регистрация признаков товаров
df_lightfm.fit_partial(items = df_merged['product_name'], item_features = goods_feature_list)

In [232]:
# Построение матрицы взаимодействий между торговыми точками и товарами
interactions, _ = df_lightfm.build_interactions(
    # Создания кортежа (торговая точка, товар, объём закупки)
    ((store, goods, quantity)
     # Перебор каждый товар
     for goods in df_matrix.index
     # Для каждого товара перебор закупок по точкам
     for store, quantity in df_matrix.loc[goods].items()
     # Учитывание только положительных закупок и товаров, которые зарегистрированы
     if quantity > 0 and goods in df_lightfm.mapping()[2])
)

In [233]:
# Формирование разреженной матрицы признаков товаров
item_features = df_lightfm.build_item_features(((row['product_name'], [str(row['product_category']) + ' ' + str(row['abc_classification'])]) for _, row in df_merged.iterrows()))

In [234]:
# Разделение матрицы взаимодействий на тренировочную и тестовую выборки
train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.5, random_state = 42)

In [235]:
# Инициализация модели с функцией потерь Warp
lightfm_model = LightFM(item_alpha = 1e-05, learning_rate = 0.01, loss = 'bpr', no_components = 16, user_alpha = 0.0)
# Обучение модели на тренировочной выборке
lightfm_model.fit(train_interactions, item_features = item_features, epochs = 10, num_threads = 2)

In [236]:
# Расчёт метрики на тестовой выборке
precision = precision_at_k(lightfm_model, test_interactions, item_features = item_features, k = 5).mean()
print("Precision@5 (тестовая выборка):", round(precision, 4))

# Расчёт метрики на тренировочной выборке
precision_ = precision_at_k(lightfm_model, train_interactions, item_features = item_features, k = 5).mean()
print("Precision@5 (тренировочная выборка):", round(precision_, 4))

Precision@5 (тестовая выборка): 0.3053
Precision@5 (тренировочная выборка): 0.388


In [237]:
# Обучение модели на всей выборке
lightfm_model.fit(interactions, item_features=item_features, epochs=10, num_threads=2)

In [238]:
# Расчёт метрики на всей выборке
precision_ = precision_at_k(lightfm_model, interactions, item_features = item_features, k = 5).mean()
print("Precision@5 (вся выборка):", round(precision_, 4))

Precision@5 (вся выборка): 0.7467


## Result

In [239]:
# Функция генерации рекомендаций товаров для торговой точки
def recommendations_goods_for_stores(store_name, product_category, number_recommendations = 5):
    # Проверка на наличие торговой точки в датасете
    if store_name not in df_matrix.columns:
        return (f'Торговая точка - {store_name} не найдена')

    # Извлечение данных о закупках по выбранной катгории товара
    store_purchases = df_matrix.loc[:, store_name].to_frame(name = 'purchased_quantity')
    store_purchases = store_purchases.reset_index()

    # Объединение данных о закупках с полными данными товаров
    merged_store = pd.merge(df_merged, store_purchases, on = 'product_name')
    # Фильтрация товаров по выбранной категории товара
    filtering_product_category = merged_store[merged_store['product_category'] == product_category]
    filtering_product_category = filtering_product_category[filtering_product_category['purchased_quantity'] > 0]

    # Проверка на наличие товаров по выбранной категории и торговой точке
    if filtering_product_category.empty:
        return (f'Нет данных о закупках в категории - {product_category} для торговой точки - {store_name}')

    # Извлечение всех товаров из выбранной категории
    candidate_products = df_merged[(df_merged['product_category'] == product_category)]
    # Лист со списком наименований товаров, которые будут рекомендоваться
    product_names = candidate_products['product_name'].values
    # Определение идентификатора магазина для модели
    store_id = df_lightfm.mapping()[0][store_name]
    # Преобразование наименований товаров в идентификаторы
    product_id = [df_lightfm.mapping()[2][p] for p in product_names if p in df_lightfm.mapping()[2]]
    # генерация рекомендаций
    generating_recommendations = lightfm_model.predict(store_id, product_id, item_features = item_features)
    # Сортировка товаров по убыванию предсказанных оценок и выбор топ-n товаров
    ratings_recommendations = np.argsort(-generating_recommendations)[:number_recommendations]
    top_product_names = [product_names[i] for i in ratings_recommendations]

    # Возвращение сгенерированных рекомендаций товаров для торговой точки
    return df_merged[df_merged['product_name'].isin(top_product_names)][['product_name', 'product_category', 'abc_classification', 'total_realization']]

In [240]:
# Приминение функции генерации рекомендаций товаров для торговой точки
recommendations_goods_for_stores('аллея', 'шоколад')

,product_name,product_category,abc_classification,total_realization
0,аленка (ок) 90г,шоколад,A,71201224.45
6,батон бабаевский с помадно-сливочной начинкой ...,шоколад,A,32311342.78
17,аленка (ок) 15г,шоколад,A,22696294.75
26,бабаевский элитный 75% какао (ок) 90г,шоколад,A,20245591.79
39,бабаевский горький (ок) 90г,шоколад,A,16489366.22
